In [12]:
import os
from openai import OpenAI
from credentials import get_credentials_openai
from pprint import pprint

In [3]:
key, project_id = get_credentials_openai()
client = OpenAI(api_key=key, project=project_id)

In [4]:
def get_completion(prompt, model='gpt-4-turbo', temperature=1, max_tokens=1000):
    '''Generates a completion for a given prompt using the OpenAI API.
    prompt: str
    model: str, optional
    temperature: float, optional
    max_tokens: int, optional
    returns: str'''
    completion = client.chat.completions.create(messages=[
            {"role": "system", "content": "you are a regular natural language user"},
            {"role": "user", "content": prompt}],
            model=model, temperature=temperature, max_tokens=max_tokens)
    return completion.choices[0].message.content

In [5]:
#Creat a path to access the data
DATAPATH = os.path.join('..', 'data')

In [6]:
#Read the statute
with open(os.path.join(DATAPATH, '18USC924c1.txt'), 'r') as f:
    statute = f.read()

In [7]:
#Case summary from the majority opinion in Smith v. United States
with open(os.path.join(DATAPATH, 'smith_case_summary.txt'), 'r') as f:
    case_summary = f.read()

In [16]:
prompt = f''''
Read the following statute: {statute}.
Now, read the following case summary: {case_summary}.
Based on this case, would you say that a reasonable person would consider that Mr. Smith _used_ 
his gun in the sense that the language of the statute requires? 
Answer in the format:
ANSWER: Your answer (yes or no) 
REASONING: Your reasoning.
Use markdown to format your answer.
'''

In [9]:
#Completion with prompt and summary from SCOTUS
first_try = get_completion(prompt)

In [13]:
pprint(first_try)

('ANSWER: Yes\n'
 '\n'
 'REASONING: According to the statute, a firearm is considered "used" during a '
 'crime of violence or drug trafficking if it contributes in any way to the '
 'criminal activity. In the case of John Angus Smith, he explicitly employed '
 'the MAC-10 firearm as part of a drug trafficking negotiation. Smith not only '
 'brought the firearm to the meeting but also introduced and presented it as a '
 'bargaining chip to trade for cocaine. This act of showcasing and offering '
 'the firearm in exchange for drugs encapsulates the statutory definition of '
 '"using" a firearm during a drug trafficking crime. The firearm was central '
 'to the negotiation and intended trade, hence making it integral to the '
 'criminal act itself, aligning with the requirements specified in the statute '
 'for using a firearm during the commission of such a crime.')


In [15]:
#Case summary from Gries and Slocum paper
case_summary = '''
John Angus Smith offered to trade an automatic weapon to an undercover officer for cocaine.
'''

In [17]:
#Completion from prompt with case summary from Gries and Slocum paper
second_try = get_completion(prompt)

In [18]:
pprint(second_try)

('ANSWER: Yes  \n'
 'REASONING:  \n'
 'The statute specifies that anyone who _uses or carries a firearm_ during and '
 'in relation to any crime of violence or drug trafficking crime shall be '
 'subject to additional penalties. In the case of John Angus Smith, he offered '
 'to trade an automatic weapon for cocaine to an undercover officer. This '
 'action demonstrates a use of the firearm during and in relation to a drug '
 'trafficking crime, as the firearm was instrumental in facilitating the '
 'illegal drug transaction. Although the weapon was not employed in a '
 'traditional violent manner, such as discharging it, the act of offering it '
 'as a trade item in a drug transaction qualifies as "using" the firearm '
 'within the context of the statute. This is because the weapon was clearly an '
 'integral part of the criminal act and served a purpose in the drug '
 'trafficking activity. Hence, this satisfies the requirement of the statute '
 'that the firearm must be used or carr

In [20]:
#Varying the temperature
answers = dict()
for temp in [0.5, 1, 1.5]:
    answers[f'completion_temp_{temp}'] = get_completion(prompt, temperature=temp)



In [22]:
for key, value in answers.items():
    print(f'Here is the completion with temperature {key}:')
    pprint(value)
    print('\n##############################################\n')

Here is the completion with temperature completion_temp_0.5:
('ANSWER: Yes  \n'
 'REASONING:  \n'
 'The statute specifies that any person who "uses or carries a firearm" during '
 'and in relation to any drug trafficking crime shall be subject to additional '
 'imprisonment. In the case of John Angus Smith, offering to trade an '
 'automatic weapon to an undercover officer for cocaine constitutes using the '
 'firearm in relation to a drug trafficking crime. The act of offering the '
 'firearm as part of a transaction for illegal drugs falls under the usage of '
 'the firearm in furtherance of committing the drug trafficking crime. '
 'Therefore, a reasonable person would consider that Mr. Smith used his gun in '
 'the sense required by the language of the statute.')

##############################################

Here is the completion with temperature completion_temp_1:
('ANSWER: Yes\n'
 '\n'
 'REASONING:\n'
 'The statute specifies that any use or carrying of a firearm in relation t

In [24]:
print(answers['completion_temp_1'])

ANSWER: Yes

REASONING:
The statute specifies that any use or carrying of a firearm in relation to a drug trafficking crime leads to additional penalties. In this scenario, John Angus Smith's act of offering an automatic weapon as a trade for cocaine directly relates his firearm to a drug trafficking transaction. Thus, the firearm was used as a means to facilitate the drug deal, fitting the statute's criteria for using a firearm in relation to a drug trafficking crime. The law does not require the firearm to be used as a weapon in the traditional sense (e.g., for intimidation or violence) to meet the definition of "use" in this context; its use as an item of barter in the crime suffices.


### All responses from GPT-4 Turbo agree with majority opinion from SCOTUS

Most interesting is this one, with standard tempreature (1.0), because it specifically deals with the question of *using* the gun as *barter*:

**ANSWER:** 
Yes 

**REASONING:**  
The statute specifies that any use or carrying of a firearm in relation to a drug trafficking crime leads to additional penalties. In this scenario, John Angus Smith's act of offering an automatic weapon as a trade for cocaine directly relates his firearm to a drug trafficking transaction. Thus, the firearm was used as a means to facilitate the drug deal, fitting the statute's criteria for using a firearm in relation to a drug trafficking crime. The law does not require the firearm to be used as a weapon in the traditional sense (e.g., for intimidation or violence) to meet the definition of "use" in this context; its use as an item of ***barter*** in the crime suffices.